In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.



In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [ ]:
game_csv = pd.read_csv('../input/Training_dataset_Original.csv',na_values=["NA", "na", "missing"])
gameDf = pd.DataFrame(game_csv)

dataDict = pd.read_csv('../input/Data_Dictionary.csv',na_values="NA")

In [ ]:
print(gameDf.shape)
print(dataDict.shape)

In [ ]:
dic = dict(zip(dataDict.values[:, 1], dataDict.values[:, 0]))

In [ ]:
#remove ID column
del gameDf[dic['Application ID (primary key)']]
del dic['Application ID (primary key)']

In [ ]:
gameX = gameDf.iloc[:,:-1]
gameY = gameDf.iloc[:,-1]

In [ ]:
range(gameDf["mvar47"].shape[0])

In [ ]:
gameX

In [ ]:
gameX.describe()

In [ ]:
#one hot encoding C=0 L=1
gameX.loc[gameX.loc[:, "mvar47"]=='C', "mvar47"] = 0
gameX.loc[gameX.loc[:, "mvar47"]=='L', "mvar47"] = 1

# gameX = np.array(gameX) 
# gameX[gameX[:, 46]=='C', 46] = 0
# gameX[gameX[:, 46]=='L', 46] = 1
# gameX = pd.DataFrame(gameX)

# gameX[gameX["mvar47"]=='C']["mvar47"] = 0
# gameX[gameX["mvar47"]=='L']["mvar47"] = 1

In [ ]:
#Missing values median
gameX.fillna(gameX.median(),inplace = True)

In [ ]:
def scoring(yhat0, y):
    
    y = np.array(y)
    
    pointsMax = 0
    thresh=0
    
    for thresh in np.arange(0.1, 0.9, 0.1):
        points = 0
        budget = 50000
        yhat = np.array(yhat0>thresh, dtype=yhat0.dtype)
        for i in range(len(y)):
            if y[i] == 0:
                if yhat[i] == 0:
                    points+=100
                    budget-=5
                else:
                    budget-=10
            else:
                if yhat[i]==0:
                    points-=50
                    budget-=10
                else:
                    points+=100
                    budget-=10
            if budget<=0:
                break
        if pointsMax<points:
            pointsMax=points
            threshMax = thresh
            
#         print(thresh, " ", points)
            
    return pointsMax, threshMax

In [ ]:
#KFold Stratified CV
from sklearn import model_selection
cv = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=100)

In [ ]:
from sklearn import model_selection

# xtrain, xtest, ytrain, ytest = model_selection.train_test_split(pokemonDf,legends,test_size=0.2,random_state=1)

import time
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

cv = model_selection.StratifiedKFold(n_splits=4, shuffle=True, random_state=100)

In [ ]:
t=time.time()

xtrain = gameX
ytrain = gameY

tempGBM=[]
finalGBM=[]
finalGBMScore=0
dMax=1
lMax=0
nMax=1
gbmMax = 0
gbmTestScore=0
gbmTrainScore=0

for d in np.arange(1,8,1):
    for l in np.arange(0.001,1,0.016):
        for n in np.arange(50,101,50):
            tempGBMScore=0
            tempGBM=[]
            for cvTrain, cvTest in cv.split(xtrain,ytrain):
                gbm=GradientBoostingClassifier(max_depth=d, learning_rate=l, n_estimators=n, random_state=100, subsample=1)
                gbm.fit(xtrain.iloc[cvTrain], ytrain.iloc[cvTrain])
                tempGBM.append(gbm)
                points, thresh = scoring(gbm.predict_proba(xtrain.iloc[cvTest])[:, 1] ,ytrain.iloc[cvTest]) 
                tempGBMScore += points
#                 scoring(gbmMax.predict_proba(gameX)[:, 1], gameY)
            tempGBMScore/=cv.n_splits
            
            print('time:', time.time()-t, "Max Score:", finalGBMScore, "thresh:", thresh ," Max l:", l, " Max n:", n, " Max d", d)
            
            if(tempGBMScore>finalGBMScore):
                finalGBMScore=tempGBMScore
                finalGBM=tempGBM
                dMax=d
                lMax=l
                nMax=n
                gbmMax = gbm


print('time:', time.time()-t)

In [ ]:
gbmMax=GradientBoostingClassifier(max_depth=2, learning_rate=2, n_estimators=2, random_state=100, subsample=1).fit(gameX, gameY)

In [ ]:
gbmMax.predict_proba(gameX).shape

In [ ]:
gbmMax.predict_proba(gameX)[0:100]

In [ ]:
gbmMax.predict(gameX)[:100]

In [ ]:
t=time.time()
score, thresh = scoring(gbmMax.predict_proba(gameX)[:, 1], gameY)
print(score, " ", thresh)
print(time.time()-t)

In [ ]:
clf = gbmMax

In [ ]:
#Leaderboard


leader_csv = pd.read_csv('../input/Leaderboard_dataset.csv',na_values=["NA", "na", "missing"])
leaderDf = pd.DataFrame(leader_csv)



leaderAppID = leaderDf['application_key']
del leaderDf['application_key']



leaderX = leaderDf

In [ ]:
#one hot encoding C=0 L=1
leaderX.loc[leaderX.loc[:, "mvar47"]=='C', "mvar47"] = 0
leaderX.loc[leaderX.loc[:, "mvar47"]=='L', "mvar47"] = 1


In [ ]:
#Missing values median
leaderX.fillna(leaderX.median(),inplace = True)

In [ ]:
pred = clf.predict_proba(leaderX)[:, 1]
pred = pd.DataFrame(pred)
pred['application_key'] = leaderAppID
pred = pred.sort_values(by = 0,kind = 'mergesort',ascending = False)

In [ ]:
# thresh = 0.6
pred = np.array(pred)
pred[pred[:, 0]>thresh, 0] = 1
pred[pred[:, 0]<=thresh, 0] = 0

In [ ]:
pred = pd.DataFrame(pred)

In [ ]:
cols = pred.columns.tolist()
cols = cols[-1:] + cols[:-1]
pred = pred[cols]
submit = pred.iloc[:10000, :]
submit = pd.DataFrame(submit)
submit[0] = 0
submit.to_csv('flabbergasted_IITGuwahati_10.csv',header = False, index = False)



In [ ]:
t=time.time()

xtrain = gameX
ytrain = gameY

tempGBM=[]
finalGBM=[]
finalGBMScore=0
dMax=1
lMax=0
nMax=1
gbmMax = 0
gbmTestScore=0
gbmTrainScore=0

for d in np.arange(1,8,1):
    for l in np.arange(0.001,1.2,0.05):
        for n in np.arange(50,99,50):
            tempGBMScore=0
            tempGBM=[]
            for cvTrain, cvTest in cv.split(xtrain,ytrain):
                gbm=GradientBoostingClassifier(max_depth=d, learning_rate=l, n_estimators=n, random_state=100, subsample=1)
                gbm.fit(xtrain.iloc[cvTrain], ytrain.iloc[cvTrain])
                tempGBM.append(gbm)
                points, thresh = scoring(gbm.predict_proba(xtrain.iloc[cvTest])[:, 1] ,ytrain.iloc[cvTest]) 
                tempGBMScore += points
#                 scoring(gbmMax.predict_proba(gameX)[:, 1], gameY)
            tempGBMScore/=cv.n_splits
            
            print('time:', time.time()-t, "Max Score:", finalGBMScore, "thresh:", thresh ," Max l:", l, " Max n:", n, " Max d", d)
            
            if(tempGBMScore>finalGBMScore):
                finalGBMScore=tempGBMScore
                finalGBM=tempGBM
                dMax=d
                lMax=l
                nMax=n
                gbmMax = gbm


print('time:', time.time()-t)